## Pre-processing for healthcare analytics

### Load Libraries

In [1]:
import pandas as pd
import os
from helpers6 import *
from past_icu_count import *
from sklearn.impute import SimpleImputer

### Load dataset

In [2]:
#Just place the dataset in the same folder as the notebook
df = pd.read_csv('master_dataset_2.csv')
df.shape

(448972, 120)

### General Preprocessing

#### Filter out missing age and gender records

In [3]:
# filter age or gender is NaN (837 records)
df = df[~df['age'].isna()]
df = df[~df['gender'].isna()]
df.shape

(448135, 120)

#### Filter out missing acuity (disabled on requested by YX)

In [4]:
# # filter out triage_acuity with null values
# df = df[df['triage_acuity'].notnull()]
# df.shape

#### Corrections to past 30/90/365d ICU stays from ED

In [5]:
df = count_n_days(df, 30)
df = count_n_days(df, 90)
df = count_n_days(df, 365)
df.shape
# no more duplicates found in this version, past ICU stays mapped using index (unique)

C:\Users\zhuyi\aha\Project Notebooks\past_icu_count.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icu.intime_ed = pd.to_datetime(df_icu.intime_ed) # convert intime to datetime type
C:\Users\zhuyi\aha\Project Notebooks\past_icu_count.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_icu.intime_icu = pd.to_datetime(df_icu.intime_icu) # convert intime to datetime type
C:\Users\zhuyi\aha\Project Notebooks\past_icu_count.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

(448135, 120)

#### Filter out patients who died before entering ed

In [6]:
# filter out rows where before_ed_mortality is 1 (18 records)
df = df[df['before_ed_mortality'] != 1]
df.shape

(448117, 120)

#### Filter out patients who died during ed

In [7]:
# filter out rows where ed_death is 1 (2 records)
df = df[df['ed_death'] != 1]
df.shape

(448115, 120)

#### Drop duplicate hospital entries on different ed entries

In [8]:
# filter rows where 'hadm_id' is null
df_null = df[df['hadm_id'].isnull()]

# filter rows where 'hadm_id' is not null
df_not_null = df[df['hadm_id'].notnull()]

# Use drop_duplicates to keep the latest occurrence of each 'hadm_id'
df_result = df_not_null.drop_duplicates(subset=['hadm_id'], keep='last')

# Combine the original DataFrame with df_not_null and the duplicates-filtered DataFrame
df = pd.concat([df_null, df_result], ignore_index=True)

print(df_null.shape, "\n", df_not_null.shape, "\n", df_result.shape, "\n", df.shape)

(234033, 120) 
 (214082, 120) 
 (213480, 120) 
 (447513, 120)


#### Drop errorneous entries for time_to_icu_transfer_hours

In [9]:
# filter rows where 'time_to_icu_transfer_hours' is null
df_null = df[df['time_to_icu_transfer_hours'].isnull()]

# filter rows where 'time_to_icu_transfer_hours' is not null
df_not_null = df[df['time_to_icu_transfer_hours'].notnull()]

# Filter the DataFrame where time_to_icu_transfer_hours >= -24
df_result = df_not_null[df_not_null['time_to_icu_transfer_hours'] >= -24]

# Combine the original DataFrame with df_not_null and the duplicates-filtered DataFrame
df = pd.concat([df_null, df_result], ignore_index=True)

df.shape


(447502, 120)

#### Drop errorneous entries for difference between 'admittime' and 'intime_ed'

In [10]:
# Convert 'admittime' and 'intime_ed' to datetime format
df['admittime'] = pd.to_datetime(df['admittime'])
df['intime_ed'] = pd.to_datetime(df['intime_ed'])

# filter rows where 'admittime' or 'admittime' is null
df_null = df[df['admittime'].isnull() | df['intime_ed'].isnull()]

# filter rows where 'admittime' or 'admittime' is not null
df_not_null = df[df['admittime'].notnull() & df['intime_ed'].notnull()]

# Filter rows where the difference between 'admittime' and 'intime_ed' >= -1 hour
df_result = df_not_null[(df_not_null['admittime'] - df_not_null['intime_ed']).dt.total_seconds() > (-3600*24)]

# Combine the original DataFrame with df_not_null and the duplicates-filtered DataFrame
df = pd.concat([df_null, df_result], ignore_index=True)

df.shape

(447444, 120)

#### Drop Features that are not used

In [11]:
# drop features deemeed unnecessary
df = df.drop(['index', 'subject_id', 'hadm_id', 'stay_id', 'anchor_age', 'anchor_year', 'dod',
              'admittime', 'dischtime', 'deathtime', 'race', 'edregtime', 'edouttime', 'in_year',
              'before_ed_mortality', 'ed_los', 'intime_icu', 'outtime_icu', 'time_to_icu_transfer', 'time_to_icu_transfer_hours',
              'next_ed_visit_time', 'next_ed_visit_time_diff', 'next_ed_visit_time_diff_days', 'chiefcomplaint'], axis=1)

df.shape

(447444, 96)

### 2. Outlier Detection

In [12]:
# from mimic-extract
vitals_valid_range = {
    'temperature': {'outlier_low': 14.2, 'valid_low': 26, 'valid_high': 45, 'outlier_high':47},
    'heartrate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 350, 'outlier_high':390},
    'resprate': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 300, 'outlier_high':330},
    'o2sat': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 100, 'outlier_high':150},
    'sbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'dbp': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 375, 'outlier_high':375},
    'pain': {'outlier_low': 0, 'valid_low': 0, 'valid_high': 10, 'outlier_high':10},
    'acuity': {'outlier_low': 1, 'valid_low': 1, 'valid_high': 5, 'outlier_high':5},
}

In [13]:
df = convert_temp_to_celcius(df)

In [14]:
df = remove_outliers(df, vitals_valid_range)
# what this really does is to transform values between valid low and outlier low to valid low,
# and values between valid high and outlier high to valid high
# anything beyond outlier low and outlier high will be set to NaN
# however the code later imputes NaN with median. i have some concerns with this and would prefer to drop the rows with NaN
# (it's approximately 12k rows only anyway)
df.shape

(447444, 96)

### 3. Data Manipulation for ML

In [15]:
# Change True and False to Binary 1 and 0
df = df.replace(True, 1)
df = df.replace(False, 0)

In [16]:
# Use the replace() method to convert 'M' to 1 and 'F' to 0 in the 'gender' column
df['gender'] = df['gender'].replace({'M': 1, 'F': 0})

In [17]:
# Define the bins and labels for age groups
# Roughly based on NIH (US) age categories, however there is no concensus on ages above 65 years as stated on NIH website
# Age Groups:
# Young Adults: 18 to 34 years
# Middle-Aged Adults: 35 to 64 years
# Older Adults: 65 to 74 years
# Elderly: 76 years and above
bins = [18, 35, 65, 75, 100]  # The age group boundaries
labels = ['Young Adults', 'Middle-Aged Adults', 'Older Adults', 'Elderly']

# Use pd.cut() to bin the 'age' column and replace values with labels
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

#### One-Hot Encodes

In [18]:
# one-hot encode Insurance Type
df = pd.get_dummies(df, columns=['insurance'])

# one-hot encode Age
df = pd.get_dummies(df, columns=['age_group'])

df.shape

(447444, 102)

In [19]:
# Check for Null Values
df.isna().sum()[df.isna().sum() > 0]

triage_temperature     25110
triage_heartrate       17963
triage_resprate        21377
triage_o2sat           21686
triage_sbp             19242
triage_dbp             20485
triage_pain            41636
triage_acuity           7313
ed_temperature_last    28265
ed_heartrate_last      19408
ed_resprate_last       19914
ed_o2sat_last          30657
ed_sbp_last            19654
ed_dbp_last            19930
ed_pain_last           52277
dtype: int64

In [20]:
# This code is to convert intime_ed from object data type to datetime data type
df['outtime_ed'] = pd.to_datetime(df['outtime_ed'])


In [21]:
# This code chunk is meant to check the data types of the columns in the data set and save them to a text file for reference.

data_types = df.dtypes

# Convert the data types to a DataFrame for better formatting
data_types_df = pd.DataFrame(data_types, columns=['Data Type'])

# Convert the DataFrame to a string
data_types_str = data_types_df.to_string()

# Save the string to a text file
with open('data_typesv3.txt', 'w') as file:
    file.write(data_types_str)

### 6. Option A: Split Train (80%) and Test (20%) data

In [22]:
# Split Train and Test data
df_train_impute=df.sample(frac=0.8,random_state=2023) #random state is a seed value
df_test_impute=df.drop(df_train_impute.index)
print(df_train_impute.shape)
print(df_test_impute.shape)

(357955, 102)
(89489, 102)


### 4. Option A: Impute NaNs with Median

In [23]:
vitals_cols = [col for col in df.columns if len(col.split('_')) > 1 and 
                                                   col.split('_')[1] in vitals_valid_range and
                                                   col.split('_')[1] != 'acuity']
vitals_cols

['triage_temperature',
 'triage_heartrate',
 'triage_resprate',
 'triage_o2sat',
 'triage_sbp',
 'triage_dbp',
 'triage_pain',
 'ed_temperature_last',
 'ed_heartrate_last',
 'ed_resprate_last',
 'ed_o2sat_last',
 'ed_sbp_last',
 'ed_dbp_last',
 'ed_pain_last']

In [24]:
imputer = SimpleImputer(strategy='median')
df_train_impute[vitals_cols] = imputer.fit_transform(df_train_impute[vitals_cols])
df_test_impute[vitals_cols] = imputer.transform(df_test_impute[vitals_cols])

### 5. Option A: Add Score values to Train data

In [25]:
add_triage_MAP(df_train_impute) # add an extra variable MAP
add_score_CCI(df_train_impute)
add_score_CART(df_train_impute)
add_score_REMS(df_train_impute)
add_score_NEWS(df_train_impute)
add_score_NEWS2(df_train_impute)
add_score_MEWS(df_train_impute)

df_train_impute.shape

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


(357955, 109)

### 5. Option A: Add Score values to Test data

In [26]:
add_triage_MAP(df_test_impute) # add an extra variable MAP
add_score_CCI(df_test_impute)
add_score_CART(df_test_impute)
add_score_REMS(df_test_impute)
add_score_NEWS(df_test_impute)
add_score_NEWS2(df_test_impute)
add_score_MEWS(df_test_impute)

df_test_impute.shape

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


(89489, 109)

### 7. Option A: Output the train and test data

In [27]:
df_train_impute.to_csv('train_impute_v3.csv', index=False)
df_test_impute.to_csv('test_impute_v3.csv', index=False)

### 4. Option B: Drop NaNs

In [28]:
df.shape

(447444, 102)

In [29]:
# drop all rows with nan values
df_drop = df.dropna()
df_drop.shape

(341290, 102)

### 5. Option B: Add Score values

In [30]:
add_triage_MAP(df_drop) # add an extra variable MAP
add_score_CCI(df_drop)
add_score_CART(df_drop)
add_score_REMS(df_drop)
add_score_NEWS(df_drop)
add_score_NEWS2(df_drop)
add_score_MEWS(df_drop)

df_drop.shape

C:\Users\zhuyi\aha\Project Notebooks\helpers6.py:472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['triage_MAP'] = df['triage_sbp']*1/3 + df['triage_dbp']*2/3
C:\Users\zhuyi\aha\Project Notebooks\helpers6.py:467: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score_CCI'] = np.select(conditions, values)
C:\Users\zhuyi\aha\Project Notebooks\helpers6.py:468: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Variable 'add_triage_MAP' successfully added
Variable 'add_score_CCI' successfully added
Variable 'Score_CART' successfully added
Variable 'Score_REMS' successfully added
Variable 'Score_NEWS' successfully added
Variable 'Score_NEWS2' successfully added
Variable 'Score_MEWS' successfully added


C:\Users\zhuyi\aha\Project Notebooks\helpers6.py:667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score_MEWS'] = np.select(conditions1, values1) + np.select(conditions2, values2) + np.select(conditions3, values3) +                              np.select(conditions4, values4)


(341290, 109)

### 6. Option B: Split Train (80%) and Test (20%) data

In [31]:
# Split Train and Test data
df_train_drop=df_drop.sample(frac=0.8,random_state=2023) #random state is a seed value
df_test_drop=df_drop.drop(df_train_drop.index)
print(df_train_drop.shape)
print(df_test_drop.shape)

(273032, 109)
(68258, 109)


### 7. Option B: Output the train and test data

In [32]:
df_train_drop.to_csv('train_drop_v3.csv', index=False)
df_test_drop.to_csv('test_drop_v3.csv', index=False)